In [94]:
import numpy as np

y_labelled = np.load('emotion_labels.npy')
X_labelled_pca = np.load('X_labelled_pca.npy')

In [95]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

class SVMClassifier:
    def __init__(self, kernel='poly', param_grid=None, cv=5, scoring='accuracy'):
        self.param_grid = param_grid or {
            'svc__degree': [2, 3, 4],
            'svc__coef0': [0, 1, 5],
            'svc__C': [0.1, 1, 10]
        }
        self.pipeline = Pipeline([
            ('svc', SVC(kernel=kernel, class_weight='balanced'))
        ])

        self.grid_search = GridSearchCV(self.pipeline, self.param_grid, cv=cv, scoring=scoring)
        self.best_model = None

    def fit(self, X, y):
        self.grid_search.fit(X, y)
        self.best_model = self.grid_search.best_estimator_
        print("Best Parameters:", self.grid_search.best_params_)
        print("Best Score:", self.grid_search.best_score_)

    def predict(self, X):
        if self.best_model is None:
            raise Exception("Model has not been trained. Call `.fit(X, y)` first.")
        return self.best_model.predict(X)
    
    def report(self, X, y_true):
        y_pred = self.predict(X)
        print("Classification Report:\n", classification_report(y_true, y_pred))


In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_labelled_pca, y_labelled, test_size=0.3, random_state=42,  stratify=y_labelled)


In [97]:
svm_clf = SVMClassifier(cv=10, param_grid = {
    'svc__C': [0.1, 1, 10, 20, 30, 40, 50, 60, 70, 80, 90,  100],       
    'svc__gamma': ['scale', 0.001, 0.01, 0.1, 1] 
}, kernel='rbf')

In [98]:
svm_clf.fit(X=X_train, y=y_train)

Best Parameters: {'svc__C': 80, 'svc__gamma': 'scale'}
Best Score: 0.8151682692307693


In [99]:
y_pred = svm_clf.predict(X=X_test)

In [100]:
svm_clf.report(X=X_test, y_true=y_test)

Classification Report:
               precision    recall  f1-score   support

       Anger       0.62      0.38      0.48        13
    Contempt       0.14      0.20      0.17         5
     Disgust       0.83      0.56      0.67        18
        Fear       1.00      0.38      0.55         8
   Happiness       0.95      0.86      0.90        21
     Neutral       0.83      0.97      0.89       178
     Sadness       0.67      0.25      0.36         8
    Surprise       0.94      0.64      0.76        25

    accuracy                           0.82       276
   macro avg       0.75      0.53      0.60       276
weighted avg       0.83      0.82      0.81       276



In [101]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_pred=y_pred, y_true=y_test)
cm

array([[  5,   0,   0,   0,   0,   8,   0,   0],
       [  0,   1,   0,   0,   0,   4,   0,   0],
       [  0,   0,  10,   0,   1,   7,   0,   0],
       [  0,   3,   0,   3,   0,   2,   0,   0],
       [  1,   0,   1,   0,  18,   1,   0,   0],
       [  1,   2,   1,   0,   0, 172,   1,   1],
       [  1,   1,   0,   0,   0,   4,   2,   0],
       [  0,   0,   0,   0,   0,   9,   0,  16]])

In [62]:
import numpy as np


unique, counts = np.unique(y_test, return_counts=True)
count_dict = dict(zip(unique, counts))

count_dict

{np.str_('Anger'): np.int64(13),
 np.str_('Contempt'): np.int64(5),
 np.str_('Disgust'): np.int64(18),
 np.str_('Fear'): np.int64(8),
 np.str_('Happiness'): np.int64(21),
 np.str_('Neutral'): np.int64(178),
 np.str_('Sadness'): np.int64(8),
 np.str_('Surprise'): np.int64(25)}